<a href="https://colab.research.google.com/github/taianers/inteligencia-artificial-gcc128/blob/master/quiz_02_traveling_salesman_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quiz 02 - Problema do Caixeiro Viajante (Traveling Salesman Problem - TSP) utilizando Algoritmos Genéticos

**GCC 128 - Inteligência Artificial**

Profº Eric Fernandes de Mello Araújo

Nome: Taiane Rodrigues de Sousa

## Objetivo

O objetivo desta tarefa é implementar um algoritmo genético para resolver o Problema do
Caixeiro Viajante (TSP). O algoritmo genético irá otimizar a ordem em que um vendedor
visita um conjunto de cidades, buscando encontrar a rota mais curta possível.

## Descrição do problema

Você recebe um conjunto de cidades e suas respectivas coordenadas em um plano
bidimensional. A tarefa é encontrar a rota mais curta possível que visite cada cidade
exatamente uma vez e retorne à cidade de origem. Esse problema é conhecido como
Problema do Caixeiro Viajante (TSP).

### Sobre a solução

É utilizada uma abordagem evolutiva, onde o objetivo é encontrar a rota mais curta que visita todas as cidades exatamente uma vez, minimizando a distância total percorrida.

1. O algoritmo começa criando uma população inicial de rotas aleatórias, representadas por permutações das cidades;
2. A população evolui através de várias gerações usando seleção por torneio, cruzamento e mutação;
3. A seleção por torneio escolhe as rotas mais aptas com base em sua distância total percorrida;
4. O cruzamento combina seguimentos de duas rotas parentais para criar rotas descendentes;
5. A mutação introduz pequenas alterações aleatórias nas rotas descendentes;
6. O algoritmo mantém o controle da melhor rota encontrada e sua distância durante as gerações;
7. Os parâmetros tamanho da população, número de gerações, taxa de cruzamento e taxa de mutação, podem ser ajustados para influenciar o comportamento e a eficácia do algoritmo (os parâmetros serão gerados de maneira aleatória para facilitar a análise a cada execução)
8. Os resultados finais exibem a melhor distância encontrada, a geração em que foi encontrada e a ordem das cidades na melhor rota.



#### Importações necessárias

- random é utlizada para gerar números aleatórios;
- math é utilizada para realizar operações matemáticas.

In [46]:
import random
import math

#### Classes Cidade e Rota

A classe `Cidade` representa a instância de uma cidade com coordenadas bidimencionais (x,y). Dentro dela foi criada uma função `calcula_distancia()` utilizando a fórmula da distância euclidiana entre dois pontos no plano cartesiano. Ela é utilizada para construir e avaliar as rotas entre as cidades.

E a classe `Rota` representa uma sequência de cidades onde a mesma calcula a distância total percorrida pela rota visitando cada cidade na ordem estabelecida. A função `calcula_distância()` é responsável por percorrer a lista de cidades e calcular a distância entre cada cidade e a próxima cidade na rota. Em geral, ela representa e avalia as rotas geradas.

In [47]:
class Cidade:
  def __init__(self, coord_x, coord_y):
    self.coord_x = coord_x
    self.coord_y = coord_y

  def calcula_distancia(self, cidade):
    dist_x = abs(self.coord_x - cidade.coord_x)
    dist_y = abs(self.coord_y - cidade.coord_y)
    dist = math.sqrt((dist_x ** 2) + (dist_y ** 2))
    return dist

In [48]:
class Rota:
  def __init__(self, cidades):
    self.cidades = cidades
    self.dist = self.calcula_distancia()

  def calcula_distancia(self):
    dist_total = 0
    num_cidades = len(self.cidades)
    for i in range(num_cidades):
      cidade_atual = self.cidades[i]
      prox_cidade = self.cidades[(i + 1) % num_cidades]
      dist_total += cidade_atual.calcula_distancia(prox_cidade)
    return dist_total

#### Função que gera rota aleatória

É responsável por gerar uma rota inicial de maneira aleatória, embaralhando a ordem das cidades. Ela utilizada na criação da população inicial garantindo a diversidade nas soluções iniciais e exploração de regiões do espaço de busca.

In [49]:
def gera_rota_aleatoria(cidades):
  rota = Rota(cidades)
  random.shuffle(rota.cidades)
  return rota

#### Funçao de seleção por torneio

Ela realiza a seleção de indivíduos mais aptos (rotas) selecionando aleatoriamente um grupo de rotas e escolhe a rota com a menor distância vencedora, ou seja, ela seleciona as rotas que serão usadas na próxima geração com base no seu desempenho. Essa estratégia permite que rotas menos aptas também tenham uma chance de serem selecionadas, auxiliando a manter a diversidade genética na população.

In [50]:
def selecao_torneio(populacao, tam_torneio):
  rotas_selecionadas = []
  for _ in range(len(populacao)):
    torneio = random.sample(populacao, tam_torneio)
    vencedor = min(torneio, key=lambda rota: rota.dist) # seleciona a rota vencedora com base na distância mínima
    rotas_selecionadas.append(vencedor)
  return rotas_selecionadas

#### Função de cruzamento (crossover)

Realiza o cruzamento entre duas rotas parentais para criar rotas descendentes. O processo é feito combinando segmentos das rotas parentais. Um ponto de corte é escolhido aleatoriamente e os segmentos são trocados entre as rotas para criar os descendentes.

In [51]:
def cruzamento(pai1, pai2):
  num_cidades = len(pai1.cidades)
  inicio = random.randint(0, num_cidades - 1)
  fim = random.randint(inicio + 1, num_cidades)

  cidades_filha = [None] * num_cidades # cria lista vazia para as cidades da rota filha
  for i in range(inicio, fim):
    cidades_filha[i] = pai1.cidades[i] # copia o segmento do pai1 para a rota filha

  cidades_restantes = [cidade for cidade in pai2.cidades if cidade not in cidades_filha] # obtém as cidades restantes do pai2 que ainda não estão na rota filha
  restante = 0
  for i in range(num_cidades):
    if cidades_filha[i] is None:
        cidades_filha[i] = cidades_restantes[restante] # preenche as posições vazias da rota filha com as cidades restantes do pai2
        restante += 1

  filha = Rota(cidades_filha)
  return filha

#### Função de mutação

Introduz pequenas alterações aleatórias na ordem das cidades dentro de uma rota, trocando aleatoriamente a posição de duas cidades na mesma. Após a troca das cidades, a função `calcula_distancia()` da classe `Rota` é chamada para atualizar a distância total. A mutação serve para evitar que o algoritmo fique preso em mínimos locais, permitindo a busca por soluções melhores.

In [52]:
def mutacao(rota, taxa):
  if random.random() < taxa:
    num_cidades = len(rota.cidades)
    indice1 = random.randint(0, num_cidades - 1)
    indice2 = random.randint(0, num_cidades - 1)
    rota.cidades[indice1], rota.cidades[indice2] = rota.cidades[indice2], rota.cidades[indice1]
    rota.dist = rota.calcula_distancia()

#### Função de aptidao (Fitness function)

Ela calcula a distância total de uma rota, levando em consideração a distância entre a última cidade e a cidade de origem. Utilizada para avaliar a qualidade das rotas identificando as que possuem menor distância e com maior probabilidade de serem selecionadas.

In [53]:
def aptidao(rota):
  dist_total = rota.dist
  dist_origem = rota.cidades[-1].calcula_distancia(rota.cidades[0])
  dist_total += dist_origem

  return dist_total

#### Função do algoritmo genérico (principal)

Ela recebe os parâmetros do problema, como a lista de cidades, o tamanho da população, o número de gerações, as taxas de cruzamento e mutação, e executa as etapas do algoritmo em um loop. A cada geração, são realizadas a seleção por torneio, o cruzamento, a mutação e a substituição da população anterior pela nova geração. A melhor rota é atualizada se uma rota com uma distância menor for encontrada.



In [54]:
def algoritmo_generico(cidades, tam_populacao, num_geracoes, tam_torneio, taxa_cruzamento, taxa_mutacao):
  # criação da população inicial de maneira aleatória
  populacao = [gera_rota_aleatoria(cidades) for _ in range(tam_populacao)]

  # variáveis para acompanhar o progresso da melhor distância e melhor geração
  melhor_dist = float('inf') # valor infinito
  melhor_geracao = 0

  print("Análise dos Resultados:")
  for geracao in range(num_geracoes):
    # seleciona as rotas mais aptas (com menor distância) da população atual
    rotas_selecionadas = selecao_torneio(populacao, tam_torneio)

    # criação de novas rotas descendentes combinando segmentos de rotas parentais
    descendentes = []
    for i in range(0, len(rotas_selecionadas), 2):
      pai1 = rotas_selecionadas[i]
      pai2 = rotas_selecionadas[i + 1] if i + 1 < len(rotas_selecionadas) else rotas_selecionadas[0]
      filha1 = cruzamento(pai1, pai2)
      filha2 = cruzamento(pai2, pai1)
      descendentes.extend([filha1, filha2])

    # gera pequenas alterações aletaórias na ordem das cidades dentro das rotas da população descendente de acordo a taxa de mutação
    for rota in descendentes:
        mutacao(rota, taxa_mutacao)

    # a população atual é substituída pela nova geração de rotas descendentes
    populacao = descendentes

    # verifica a melhor distância, se a atual for menor que a melhor encontrada até o momento, é atualizada como melhor juntamente com o valor da melhor geração
    melhor_rota = min(populacao, key=aptidao)
    melhor_dist_atual = aptidao(melhor_rota)
    if melhor_dist_atual < melhor_dist:
      melhor_dist = melhor_dist_atual
      melhor_geracao = geracao + 1

  # imprime os resultados
  print(f"\nMelhor distância: {melhor_dist}")
  print(f"Geração em que foi encontrada: {melhor_geracao}")
  print("\nMelhor rota:")
  for cidade in melhor_rota.cidades:
      print(f"Cidade ({cidade.coord_x}, {cidade.coord_y})")

  return melhor_rota

# exemplo de representação de cidades
cidades = [
    Cidade(0, 0),
    Cidade(1, 5),
    Cidade(5, 2),
    Cidade(8, 3),
    Cidade(6, 8)
]

# parâmetros gerados modo aleatório
tam_populacao = random.randint(50, 200)
num_geracoes = random.randint(500, 2000)
tam_torneio = random.randint(2, 10)
taxa_cruzamento = random.uniform(0.6, 1.0)
taxa_mutacao = random.uniform(0.01, 0.1)

print("Valores dos parâmetros:\n")
print(f"Tamanho da população: {tam_populacao}")
print(f"Número de gerações: {num_geracoes}")
print(f"Tamanho do torneio: {tam_torneio}")
print(f"Taxa de cruzamento: {taxa_cruzamento}")
print(f"Taxa de mutação: {taxa_mutacao}\n")

melhor_rota = algoritmo_generico(cidades, tam_populacao, num_geracoes, tam_torneio, taxa_cruzamento, taxa_mutacao)

print("\nDistância da melhor rota:", aptidao(melhor_rota))

Valores dos parâmetros:

Tamanho da população: 68
Número de gerações: 1486
Tamanho do torneio: 2
Taxa de cruzamento: 0.6773322127622738
Taxa de mutação: 0.07451878558810995

Análise dos Resultados:

Melhor distância: 28.024856343043847
Geração em que foi encontrada: 6

Melhor rota:
Cidade (6, 8)
Cidade (1, 5)
Cidade (0, 0)
Cidade (5, 2)
Cidade (8, 3)

Distância da melhor rota: 30.247743490009977


#### Discussões

- Eficácia:
  - Para 5 execuções, foi possível analisar que o algoritmo foi eficaz em encontrar soluções próximas ou iguais a melhor solução da distância mínima para o problema do caixeiro viajante. Cada resultado mostrou uma rota com uma distância relativamente próxima à melhor distância possível de acordo a variação dos parâmetros.

- Convergência:
  - Nas 5 execuções, é observado que o algoritmo genérico demonstrou um comportamento de convergência rápido e eficiente para encontrar soluções de alta qualidade para o problema. Em maior parte dos casos a melhor solução foi encontrada entre as primeiras gerações, indicando uma rápida convergência para soluções ótimas ou quase ótimas.

Em geral, podemos perceber que o comportamento pode variar dependendo dos valores dos parâmetros.

- Resultados utilizados para análise, abaixo:

```
- Resultado 1:
Valores dos parâmetros:

Tamanho da população: 181
Número de gerações: 922
Tamanho do torneio: 7
Taxa de cruzamento: 0.9243344057993589
Taxa de mutação: 0.042562170499659026

Análise dos Resultados:

Melhor distância: 28.024856343043847
Geração em que foi encontrada: 2

Melhor rota:
Cidade (5, 2)
Cidade (0, 0)
Cidade (1, 5)
Cidade (6, 8)
Cidade (8, 3)

Distância da melhor rota: 28.024856343043847

- Resultado 2:
Valores dos parâmetros:

Tamanho da população: 200
Número de gerações: 1223
Tamanho do torneio: 9
Taxa de cruzamento: 0.9483597356838618
Taxa de mutação: 0.022604530627940038

Análise dos Resultados:

Melhor distância: 30.69353057772077
Geração em que foi encontrada: 1

Melhor rota:
Cidade (1, 5)
Cidade (0, 0)
Cidade (5, 2)
Cidade (8, 3)
Cidade (6, 8)

Distância da melhor rota: 30.69353057772077

- Resultado 3:
Valores dos parâmetros:

Tamanho da população: 147
Número de gerações: 982
Tamanho do torneio: 4
Taxa de cruzamento: 0.775944852815273
Taxa de mutação: 0.08896397695649186

Análise dos Resultados:

Melhor distância: 29.96159819646826
Geração em que foi encontrada: 1

Melhor rota:
Cidade (5, 2)
Cidade (8, 3)
Cidade (6, 8)
Cidade (1, 5)
Cidade (0, 0)

Distância da melhor rota: 30.247743490009977

- Resultado 4:
Valores dos parâmetros:

Tamanho da população: 90
Número de gerações: 1005
Tamanho do torneio: 5
Taxa de cruzamento: 0.9430867966688379
Taxa de mutação: 0.06710494850504752

Análise dos Resultados:

Melhor distância: 28.024856343043847
Geração em que foi encontrada: 3

Melhor rota:
Cidade (5, 2)
Cidade (0, 0)
Cidade (1, 5)
Cidade (6, 8)
Cidade (8, 3)

Distância da melhor rota: 28.024856343043847

- Resultado 5:

Valores dos parâmetros:

Tamanho da população: 104
Número de gerações: 1618
Tamanho do torneio: 4
Taxa de cruzamento: 0.9801068473233547
Taxa de mutação: 0.03405363063167091

Análise dos Resultados:

Melhor distância: 28.024856343043847
Geração em que foi encontrada: 11

Melhor rota:
Cidade (8, 3)
Cidade (6, 8)
Cidade (1, 5)
Cidade (0, 0)
Cidade (5, 2)

Distância da melhor rota: 28.024856343043847
```

